In [1]:
import recognitionTypes.hardCodedRecognition as hard_coded
from recognitionTypes.classificatorsRecognition import ClassificatorRecognition as classificator
from recognitionTypes.neuralNetworkRecognition import NeuralRecognition as neuralRecognition
import cv2 as cv2
import mediapipe as mp
import pandas as pd

In [2]:
neural = neuralRecognition()
neural.trainNetwork(True)

Epoch 1/11
 96/136 [====================>.........] - ETA: 0s - loss: 2.1168 - accuracy: 0.2563
Epoch 1: val_loss improved from inf to 1.51659, saving model to keypoint_classifier\keypoint_classifier.hdf5
136/136 [==============================] - 1s 2ms/step - loss: 1.9894 - accuracy: 0.3124 - val_loss: 1.5166 - val_accuracy: 0.5268
Epoch 2/11
 94/136 [===================>..........] - ETA: 0s - loss: 1.1880 - accuracy: 0.5592
Epoch 2: val_loss improved from 1.51659 to 0.72942, saving model to keypoint_classifier\keypoint_classifier.hdf5
136/136 [==============================] - 0s 2ms/step - loss: 1.0698 - accuracy: 0.6035 - val_loss: 0.7294 - val_accuracy: 0.7502
Epoch 3/11
 94/136 [===================>..........] - ETA: 0s - loss: 0.6158 - accuracy: 0.7872
Epoch 3: val_loss improved from 0.72942 to 0.48192, saving model to keypoint_classifier\keypoint_classifier.hdf5
136/136 [==============================] - 0s 2ms/step - loss: 0.5843 - accuracy: 0.8025 - val_loss: 0.4819 - val_a

In [3]:
def mainprogram(recognitionType):
    cap = cv2.VideoCapture(0)

    mpHands = mp.solutions.hands
    hands = mpHands.Hands( max_num_hands=1, min_detection_confidence=0.9, static_image_mode=False)
    mpDraw = mp.solutions.drawing_utils

    while True:
        success, img = cap.read()
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img,1)
        results = hands.process(img)
        h, w, c = img.shape
        cv2.rectangle(img, (20,20), (300,400), (255,0,0), 2)

        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                for id, lm in enumerate(handLms.landmark):
                    cx, cy = int(lm.x * w), int(lm.y * h)

                landMarkVect = handLms.landmark
                hard_coded.checkPointsRange(landMarkVect, w, h)

                gestureName = recognitionType.getGestureName(landMarkVect, w, h)

                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
                cv2.putText(img, gestureName, (10, 70), cv2.FONT_ITALIC, 1, (0, 0, 255), 1)

        cv2.imshow("Image", img)
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            cv2.destroyAllWindows()
            break

In [4]:
# mainprogram(neural)